# Config Tool

The `uwtools` API's `config` module provides functions to create and manipulate configuration files, objects, and dictionaries.

<div class="alert alert-warning"><b>Note: </b>This notebook was tested using <code>uwtools</code> version 2.6.0.</div>
<div class="alert alert-info">For more information, please see the <a href="https://uwtools.readthedocs.io/en/2.5.0/sections/user_guide/api/config.html">uwtools.api.config</a> Read the Docs page.</div>

## Table of Contents

* [Getting Config Objects](#Getting-Config-Objects)
* [Config Depth Limitations](#Config-Depth-Limitations)
* [Realizing Configs](#Realizing-Configs)
    * [Updating Configs](#Updating-Configs)
    * [Using the `key_path` Parameter](#Using-the-key_path-Parameter)
    * [Using the `values_needed` Parameter](#Using-the-values_needed-Parameter)
    * [Using the `total` Parameter](#Using-the-total-Parameter)
* [Realizing Configs to a Dictionary](#Realizing-Configs-to-a-Dictionary)
* [Comparing Configs](#Comparing-Configs)
* [Validating Configs](#Validating-Configs)
* [Working with Config Classes](#Working-with-Config-Classes)
    * [Comparing Config Objects](#Comparing-Config-Objects)
    * [Rendering Values](#Rendering-Values)
    * [Writing Configs in a Specified Format](#Writing-Configs-in-a-Specified-Format)
    * [Updating Values](#Updating-Values) 
<!--cell 0-->

In [1]:
import os
from pathlib import Path
from freezegun import freeze_time
from uwtools.api import config
from uwtools.api.logging import use_uwtools_logger

if os.environ.get("FAKE_NOW"):
    freeze_time("2000-01-01 00:00:00 UTC").start()

use_uwtools_logger()

## Getting Config Objects

The `config` tool can create configuration objects given a Python ``dict`` or a file in one of five different formats: FieldTable, INI, Fortran namelist, Shell, or YAML. `config.get_yaml_config` is demonstrated here, but the config module also has similar functions for each of the other supported formats: `get_fieldtable_config()`,  `get_ini_config()`, `get_nml_config()`, and `get_sh_config()`.
<!--cell 2-->

In [2]:
help(config.get_yaml_config)

Help on function get_yaml_config in module uwtools.api.config:

get_yaml_config(config: 'dict | Path | str | None' = None, stdin_ok: 'bool' = False) -> 'YAMLConfig'
    Get a ``YAMLConfig`` object.
    
    :param config: YAML file or ``dict`` (``None`` => read ``stdin``).
    :param stdin_ok: OK to read from ``stdin``?
    :return: An initialized ``YAMLConfig`` object.



The `stdin_ok` argument can be used to permit reads from `stdin`, but this is a rare use case beyond the scope of this notebook that will not be discussed here.

`get_yaml_config()` can take input from a Python `dict` or a YAML file like the one below.
<!--cell 4-->

In [3]:
%%bash
cat fixtures/config/get-config.yaml

greeting: Hello
recipient: World


Paths to config files can be provided either as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Since `get_yaml_config()` is used here, a `YAMLConfig` object is returned.
<!--cell 6-->

In [4]:
config1 = config.get_yaml_config(
    config=Path("fixtures/config/get-config.yaml")
)
print(type(config1))
print(config1)

<class 'uwtools.config.formats.yaml.YAMLConfig'>
greeting: Hello
recipient: World


Providing a Python `dict` will create a UW `Config` object with format matching the function used.
<!--cell 8-->

In [5]:
input_config = {"message": {"greeting":"Hi", "recipient":"Earth"}}
config2 = config.get_yaml_config(
    config=input_config
)
print(config2)

message:
  greeting: Hi
  recipient: Earth


## Config Depth Limitations

Some config formats have limitations on the depth of their nested configs. Shell configs, for example, may only contain top-level, bash-syntax `key=value` pairs.
<!--cell 10-->

In [6]:
config.get_sh_config(
    config={"greeting":"Salutations", "recipient":"Mars"}
)

greeting=Salutations
recipient=Mars

Shell configs cannot be nested, and any attempt to do so will raise a `UWConfigError`.
<!--cell 12-->

In [7]:
try: 
    config.get_sh_config(
        config={"message": {"greeting":"Salutations", "recipient":"Mars"}}
    )
except Exception as e:
    print(e)

Cannot instantiate SHConfig from depth-2 config


When creating INI configs, exactly one level of nesting is required so that each key-value pair is contained within a section. The top level keys become sections, which are contained within square brackets `[]`. Read more about INI configuration files <a href="https://en.wikipedia.org/wiki/INI_file">here</a>.
<!--cell 14-->

In [8]:
config.get_ini_config(
    config={"message": {"greeting":"Salutations", "recipient":"Mars"}}
)

[message]
greeting = Salutations
recipient = Mars

Either more or fewer levels of nesting will raise a `UWConfigError`.
<!--cell 16-->

In [9]:
try:
    config.get_ini_config(
        config={"greeting":"Salutations", "recipient":"Mars"}
    )
except Exception as e:
    print(e)

Cannot instantiate INIConfig from depth-1 config


## Realizing Configs

The `config.realize()` function writes config files to disk or `stdout` with the ability to render Jinja2 expressions and add/update values.
<!--cell 18-->

In [10]:
help(config.realize)

Help on function realize in module uwtools.api.config:

realize(input_config: 'Config | Path | dict | str | None' = None, input_format: 'str | None' = None, update_config: 'Config | Path | dict | str | None' = None, update_format: 'str | None' = None, output_file: 'Path | str | None' = None, output_format: 'str | None' = None, key_path: 'list[YAMLKey] | None' = None, values_needed: 'bool' = False, total: 'bool' = False, dry_run: 'bool' = False, stdin_ok: 'bool' = False) -> 'dict'
    Realize a config based on a base input config and an optional update config.
    
    The input config may be specified as a filesystem path, a ``dict``, or a ``Config`` object. When it
    is not, it will be read from ``stdin``.
    
    If an update config is specified, it is merged onto the input config, augmenting or overriding base
    values. It may be specified as a filesystem path, a ``dict``, or a ``Config`` object. When it is
    not, it will be read from ``stdin``.
    
    At most one of the in

The `input_config` parameter takes a config from a string path, <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object, Python `dict`, or UW `Config` object like the `YAMLConfig` object from the <a href="#Getting-Config-Objects">Getting Config Objects</a> section. The `input_format` argument must be provided for `dict` inputs or for files without recognized extensions. Configs are written to `stdout` if `output_file` is unspecified or explicitly set to `None`, or to the file specified by `output_file`. The `output_format` argument must be provided when writing to `stdout` or to a file without a recognized extension. Recognized extensions are: `.ini`, `.nml`, `.sh`, and `.yaml`.
<!--cell 20-->

In [11]:
config.realize(
    input_config=config1,
    output_file=Path('tmp/config1.yaml')
)

{'greeting': 'Hello', 'recipient': 'World'}

The `realize()` method returns a dict version of the config regardless of input type, and the file is written in the YAML format as indicated by the file extension.
<!--cell 22-->

In [12]:
%%bash
cat tmp/config1.yaml

greeting: Hello
recipient: World


Input and output formats are not required to match. This can be used to convert some configs from one format to another. YAML configs can be converted to configs of other recognized formats so long as the depth restrictions of the output format are met. All configs of recognized formats can be converted into YAML configs. Keep in mind that some formats are unable to express some types (for example, Shell configs can't express a value as an `int` while a Fortran namelist can) so type information may be lost when converting between formats.
<!--cell 24-->

In [13]:
config.realize(
    input_config='fixtures/config/get-config.yaml',
    input_format='yaml',
    output_file='tmp/realize-config.sh',
    output_format='sh'
)

{'greeting': 'Hello', 'recipient': 'World'}

Here a Shell config is created from a YAML config.
<!--cell 26-->

In [14]:
%%bash
cat tmp/realize-config.sh

greeting=Hello
recipient=World


### Updating Configs

Configs can be updated by providing a second config with the `update_config` parameter. If the update config contains keys that match the base config, the base config values for those keys will be overwritten. Once updated, if the config contains Jinja2 expressions, like the one below, they will be rendered in the config wherever possible.
<!--cell 28-->

In [15]:
%%bash
cat fixtures/config/base-config.nml

&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .FALSE.
/


Here, the update config provides values that will update two of the Jinja2 expressions and override one key with a new value.
<!--cell 30-->

In [16]:
config.realize(
    input_config='fixtures/config/base-config.nml',
    update_config={"memo": {"greeting":"Salutations", "recipient":"Mars", "sent": True}},
    output_file='tmp/updated-config.nml'
)

{'memo': {'sender_id': '{{ id }}',
  'message': 'Salutations, Mars!',
  'sent': True,
  'greeting': 'Salutations',
  'recipient': 'Mars'}}

All of the key-value pairs were added to the updated config, and the base config was rendered where the appropriate values were provided. However, not all Jinja2 expressions are required to be rendered: An `id` key was not provided in the update config, so the expression referencing it was not rendered.
<!--cell 32-->

In [17]:
%%bash
cat tmp/updated-config.nml

&memo
    sender_id = '{{ id }}'
    message = 'Salutations, Mars!'
    sent = .true.
    greeting = 'Salutations'
    recipient = 'Mars'
/


### Using the `key_path` Parameter

Consider the following config file, where the desired keys and values are not at the top level.
<!--cell 34-->

In [18]:
%%bash
cat fixtures/config/keys-config.yaml

keys:
  to:
    config:
      message: "{{ greeting }}, {{ recipient }}!"


The `key_path` parameter allows only a portion of the config, identified by following a given list of keys, to be written to a file or, in this case, to `stdout`. Note that the key-value pairs from the update config are used to render values, but don't appear in the config written to `stdout`.
<!--cell 36-->

In [19]:
_ = config.realize(
    input_config="fixtures/config/keys-config.yaml",
    update_config={"greeting": "Good morning", "recipient": "Venus"},
    output_file=None,
    output_format='yaml',
    key_path=['keys', 'to', 'config']
)

message: Good morning, Venus!


### Using the `values_needed` Parameter

Consider the config file below, which contains unrendered Jinja2 expressions.
<!--cell 38-->

In [20]:
%%bash
cat fixtures/config/base-config.nml

&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .FALSE.
/


Setting `values_needed` to `True` will allow logging of keys that contain unrendered Jinja2 expressions and their values. A logger needs to be initialized for this information to be displayed. The config is not written and the returned `dict` is empty.
<!--cell 40-->

In [21]:
config.realize(
    input_config='fixtures/config/base-config.nml',
    output_file=None,
    output_format='nml',
    values_needed=True
)

[2000-01-01T00:00:00]     INFO Keys that are complete:
[2000-01-01T00:00:00]     INFO   memo
[2000-01-01T00:00:00]     INFO   memo.sent
[2000-01-01T00:00:00]     INFO 
[2000-01-01T00:00:00]     INFO Keys with unrendered Jinja2 variables/expressions:
[2000-01-01T00:00:00]     INFO   memo.sender_id: {{ id }}
[2000-01-01T00:00:00]     INFO   memo.message: {{ greeting }}, {{ recipient }}!


{}

### Using the `total` Parameter

The `total` parameter is used to specify that all Jinja2 expressions must be rendered before the final config is written. Consider the config below which contains multiple expressions.
<!--cell 42-->

In [22]:
%%bash
cat fixtures/config/base-config.nml

&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .FALSE.
/


As was shown in the <a href="#Updating-Configs">Updating Configs</a> section, by default not all Jinja2 expressions are required to be rendered. However, when `total` is set to `True` and not enough values are provided to fully realize the config, a `UWConfigRealizeError` is raised. Notice that values are provided for `greeting` and `recipient`, but not for `id`.
<!--cell 44-->

In [23]:
try:
    config.realize(
        input_config='fixtures/config/base-config.nml',
        update_config={"memo": {"greeting":"Salutations", "recipient":"Mars", "sent":True}},
        output_file='tmp/config-total.nml',
        total=True
    )
except Exception as e:
    print(e)

Config could not be totally realized


With all values provided to fully render the config, `realize()` writes the complete config without error.
<!--cell 46-->

In [24]:
config.realize(
    input_config='fixtures/config/base-config.nml',
    update_config={"memo": {"greeting":"Salutations", "recipient":"Mars", "sent":True, "id":321}},
    output_file='tmp/config-total.nml',
    total=True
)

{'memo': {'sender_id': '321',
  'message': 'Salutations, Mars!',
  'sent': True,
  'greeting': 'Salutations',
  'recipient': 'Mars',
  'id': 321}}

The newly created config file is free from any unrendered Jinja2 expressions.
<!--cell 48-->

In [25]:
%%bash
cat tmp/config-total.nml

&memo
    sender_id = '321'
    message = 'Salutations, Mars!'
    sent = .true.
    greeting = 'Salutations'
    recipient = 'Mars'
    id = 321
/


## Realizing Configs to a Dictionary

The `config.realize_to_dict()` function has the ability to manipulate config values, and returns the config as a Python `dict` just as `realize()` does. However, a config won't be written to a file or to `stdout`. Like `realize()`, input or update configs can be Python dictionaries, UW `Config` objects, or files like the one below.
<!--cell 50-->

In [26]:
%%bash
cat fixtures/config/get-config.yaml

greeting: Hello
recipient: World


`realize_to_dict()` has the same parameters as `realize()`, with the exception of `output_file` and `output_format`. Instead, configs can be manipulated or converted to a `dict` without the need to specify an output file or format.
<!--cell 52-->

In [27]:
config.realize_to_dict(
    input_config={"id": "456"},
    update_config="fixtures/config/get-config.yaml"
)

{'id': '456', 'greeting': 'Hello', 'recipient': 'World'}

For more details on usage and parameters, see the <a href="#Realizing-Configs">Realizing Configs</a> section above.

## Comparing Configs

The `config` tool can be used to compare two configuration files using `config.compare()`.
<!--cell 54-->

In [28]:
help(config.compare)

Help on function compare in module uwtools.api.config:

compare(path1: 'Path | str', path2: 'Path | str', format1: 'str | None' = None, format2: 'str | None' = None) -> 'bool'
    Compare two config files.
    
    Recognized file extensions are: ``ini``, ``nml``, ``sh``, ``yaml``
    
    :param path1: Path to 1st config file.
    :param path2: Path to 2nd config file.
    :param format1: Format of 1st config file (optional if file's extension is recognized).
    :param format2: Format of 2nd config file (optional if file's extension is recognized).
    :return: ``False`` if config files had differences, otherwise ``True``.



Consider the following config files, which have similar values, with the exception of `sent`'s value.
<!--cell 56-->

In [29]:
%%bash
cat fixtures/config/base-config.nml
echo ----------------------------------------------
cat fixtures/config/alt-config.nml 

&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .FALSE.
/
----------------------------------------------
&memo
  sender_id = "{{ id }}"
  message = "{{ greeting }}, {{ recipient }}!"
  sent = .TRUE.
/


`compare()` returns `True` if the configs contain identical key-value pairs, and `False` otherwise. If a logger has been initialized, information is logged on which files are being compared and the values that differ, if any. Files are passed to `path1` and `path2` as a string filename or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Corresponding optional formats may be passed using `format1` and `format2` and are only needed if the format suffix is not recognized.
<!--cell 58-->

In [30]:
config.compare(
    path1=Path('fixtures/config/base-config.nml'),
    path2='fixtures/config/alt-config.nml',
    format1='nml',
    format2='nml'
)

[2000-01-01T00:00:00]     INFO - fixtures/config/base-config.nml
[2000-01-01T00:00:00]     INFO + fixtures/config/alt-config.nml
[2000-01-01T00:00:00]     INFO ---------------------------------------------------------------------
[2000-01-01T00:00:00]     INFO ↓ ? = info | -/+ = line unique to - or + file | blank = matching line
[2000-01-01T00:00:00]     INFO ---------------------------------------------------------------------
[2000-01-01T00:00:00]     INFO   memo:
[2000-01-01T00:00:00]     INFO     message: '{{ greeting }}, {{ recipient }}!'
[2000-01-01T00:00:00]     INFO     sender_id: '{{ id }}'
[2000-01-01T00:00:00]     INFO -   sent: false
[2000-01-01T00:00:00]     INFO +   sent: true


False

To see the behavior of `compare()` when key-value pairs are identical, one of the configs from above is copied in the cell below.
<!--cell 60-->

In [31]:
%%bash
cp fixtures/config/base-config.nml tmp/config-copy.nml

When these two files are compared, `True` is returned and the log reports no differences.
<!--cell 62-->

In [32]:
config.compare(
    path1='fixtures/config/base-config.nml',
    path2='tmp/config-copy.nml',
)

[2000-01-01T00:00:00]     INFO - fixtures/config/base-config.nml
[2000-01-01T00:00:00]     INFO + tmp/config-copy.nml


True

If a comparison is attempted between two files whose formats that don't match, `compare()` returns `False` and the mismatch is reported.
<!--cell 64-->

In [33]:
config.compare(
    path1=Path('fixtures/config/get-config.yaml'),
    path2=Path('fixtures/config/base-config.nml')
)

[2000-01-01T00:00:00]    ERROR Formats do not match: yaml vs nml


False

## Validating Configs

The `config.validate()` function checks if a given config conforms to a specified JSON schema.
<!--cell 66-->

In [34]:
help(config.validate)

Help on function validate in module uwtools.api.config:

validate(schema_file: 'Path | str', config_data: 'ConfigDataT | None' = None, config_path: 'ConfigPathT | None' = None, stdin_ok: 'bool' = False) -> 'bool'
    Check whether the specified config conforms to the specified JSON Schema spec.
    
    Specify at most one of config_data or config_path. If no config is specified, ``stdin`` is read
    and will be parsed as YAML and then validated.
    
    :param schema_file: The JSON Schema file to use for validation.
    :param config_data: A config to validate.
    :param config_path: A path to a file containing a config to validate.
    :param stdin_ok: OK to read from ``stdin``?
    :raises: TypeError if both config_* arguments specified.
    :return: ``True`` if the YAML file conforms to the schema, ``False`` otherwise.



Consider the simple YAML config below. `validate()` used together with an appropriate JSON schema ensures that the config meets expectations before it's used elsewhere.
<!--cell 68-->

In [35]:
%%bash
cat fixtures/config/get-config.yaml

greeting: Hello
recipient: World


Below is an example of a schema used to validate a config. It ensures that the required keys are present and the value types match expectations. For information on the keys used here and more, please refer to <a href="https://json-schema.org/docs">JSON Schema documentation</a>.
<!--cell 70-->

In [36]:
%%bash
cat fixtures/config/validate.jsonschema

{
  "additionalProperties": false,
  "properties": {
    "greeting": {
      "type": "string"
    },
    "recipient": {
      "type": "string"
    }
  },
  "required": [
    "greeting", "recipient"
  ],
  "type": "object"
}


The schema file and config from above are passed to the respective `schema_file` and `config_path` parameters. Config file paths should be passed as a string or <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. Files should be of YAML format, or parseable as YAML. Alternatively, a `YAMLConfig` object or a Python `dict` can be provided. `validate()` returns `True` if the config conforms to the JSON schema, and `False` otherwise. With a logger initialized, details about any validation errors are reported.
<!--cell 72-->

In [37]:
config.validate(
    schema_file='fixtures/config/validate.jsonschema',
    config_path='fixtures/config/get-config.yaml'
)

[2000-01-01T00:00:00]     INFO Schema validation succeeded for config


True

A mutually-exclusive alternative to the `config_path` argument, the `config_data` argument commonly accepts a `dict` object, but can also validate configs based on `bool`, `float`, `int`, `list`, or `str` values. In the next example, validation errors exist, and the logger reports the number of errors found along with their locations and details.
<!--cell 74-->

In [38]:
config.validate(
    schema_file='fixtures/config/validate.jsonschema',
    config_data={'greeting':'Hello', 'recipient':47}
)

[2000-01-01T00:00:00]    ERROR 1 schema-validation error found in config
[2000-01-01T00:00:00]    ERROR Error at recipient:
[2000-01-01T00:00:00]    ERROR   47 is not of type 'string'


False

## Working with Config Classes

The `config` tool provides five classes that can be used to work with configs in an object-oriented way. The five different classes each work with a single format: `config.FieldTableConfig`, `config.INIConfig`, `config.NMLConfig`, `config.SHConfig`, and `config.YAMLConfig`. `config.INIConfig` is demonstrated here, but the other classes all use methods of the same names for working with each respective format.
<!--cell 76-->

In [39]:
help(config.INIConfig)

Help on class INIConfig in module uwtools.config.formats.ini:

class INIConfig(uwtools.config.formats.base.Config)
 |  INIConfig(config: 'dict | Path | None' = None)
 |  
 |  Work with INI configs.
 |  
 |  Method resolution order:
 |      INIConfig
 |      uwtools.config.formats.base.Config
 |      abc.ABC
 |      collections.UserDict
 |      collections.abc.MutableMapping
 |      collections.abc.Mapping
 |      collections.abc.Collection
 |      collections.abc.Sized
 |      collections.abc.Iterable
 |      collections.abc.Container
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, config: 'dict | Path | None' = None)
 |      :param config: Config file to load (None => read from stdin), or initial dict.
 |  
 |  as_dict(self) -> 'dict'
 |      Returns a pure dict version of the config.
 |  
 |  dump(self, path: 'Path | None' = None) -> 'None'
 |      Dump the config in INI format.
 |      
 |      :param path: Path to dump config to (default: stdout).
 | 

An object can be initialized by providing a config either as a Python `dict` or a <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> to the file.
<!--cell 78-->

In [40]:
fruits = config.INIConfig(
    config=Path('fixtures/config/fruit-config.ini')
)
print(fruits)

[fruit count]
apples = 3
grapes = {{ grape_count }}
kiwis = 2


### Comparing Config Objects

The `compare_config()` method compares two config `dict`s and returns `True` when they match and `False` otherwise. Two config `dict`s can be passed to the `dict1` and `dict2` parameters. Config objects of every format use the same method demonstrated here, and it stands as an alternative to `config.compare()`, which compares files rather than dictionaries. See the [Comparing Configs](#Comparing-Configs) section above for more details on `config.compare()`. The configs compared using `compare_config()` can be compared without regard for their intended format, since they are compared as dictionaries, but they must have a section/key/value structure.
<!--cell 80-->

In [41]:
fruits.compare_config(
    dict1={'fruit count':{'apples':'3', 'grapes':'8', 'kiwis':'1'}},
    dict2={'fruit count':{'apples':'3', 'grapes':'8', 'kiwis':'1'}}
)

True

If `dict2` is left unspecified or set to `None`, the `dict1` config is compared to the config stored in the object itself. When there are differences between the two configs, as is the case here, `False` is returned. When a logger is initialized, the values that differ are displayed.
<!--cell 82-->

In [42]:
fruits.compare_config(
    dict1={'fruit count':{'apples':'3', 'grapes':'8', 'kiwis':'1'}}
)

[2000-01-01T00:00:00]     INFO ---------------------------------------------------------------------
[2000-01-01T00:00:00]     INFO ↓ ? = info | -/+ = line unique to - or + file | blank = matching line
[2000-01-01T00:00:00]     INFO ---------------------------------------------------------------------
[2000-01-01T00:00:00]     INFO   fruit count:
[2000-01-01T00:00:00]     INFO     apples: '3'
[2000-01-01T00:00:00]     INFO -   grapes: '{{ grape_count }}'
[2000-01-01T00:00:00]     INFO +   grapes: '8'
[2000-01-01T00:00:00]     INFO -   kiwis: '2'
[2000-01-01T00:00:00]     INFO ?           ^
[2000-01-01T00:00:00]     INFO +   kiwis: '1'
[2000-01-01T00:00:00]     INFO ?           ^


False

### Rendering Values

If the object's config contains unrendered Jinja2 expressions, the `dereference()` method will render as many as possible. The optional `context` parameter can be used to provide additional values with a Python `dict`.
<!--cell 84-->

In [43]:
fruits.dereference(
    context={'grape_count':'15'}
)
print(fruits)

[fruit count]
apples = 3
grapes = 15
kiwis = 2


### Writing Configs in a Specified Format

Each of the `config` tool's classes provide methods that write configs of their format. With the `fruits` object, which is an instance of `INIConfig`, INI configs are written. `dump()` is one of these methods, which writes the config stored in the object to a file specified by providing the `path` parameter with a <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object. If `path` is `None`, the config is written to `stdout`.
<!--cell 86-->

In [44]:
fruits.dump(
    path=Path('tmp/fruits.ini')
)

Below we can see that the config was written in the INI format at the specified path.
<!--cell 88-->

In [45]:
%%bash
cat tmp/fruits.ini

[fruit count]
apples = 3
grapes = 15
kiwis = 2


To write a config that is not stored in the object, the `dump_dict()` method is used. This method takes a config in the form of a `dict` and, like `dump()`, writes the config in the INI format to `stdout` if `path` is `None` or to the path that a <a href="https://docs.python.org/3/library/pathlib.html#pathlib.Path">Path</a> object indicates.
<!--cell 90-->

In [46]:
other_fruits = {'fruit count':{'oranges':4, 'blueberries':9}}
fruits.dump_dict(
    cfg=other_fruits,
    path=None
)

[fruit count]
oranges = 4
blueberries = 9


### Updating Values

The `update_from()` method adds new or updated key-value pairs to the stored config, and these are provided as a dictionary via the `src` parameter. 
<!--cell 92-->

In [47]:
fruits.update_from(
    src={'fruit count':{'kiwis': '4', 'raspberries': '12'}}
)
print(fruits)

[fruit count]
apples = 3
grapes = 15
kiwis = 4
raspberries = 12
